# NOAA 1158: Setup Heating Models
Simulation of loops in NOAA 1158 in which all of the strands are heated by nanoflares varying frequency. In this notebook, we will setup models for several different heating frequencies, likely low, intermediate, and high.

For all of the models created here, we will use the base AR model of NOAA 1158 that we've already built so that the geometry is consistent across all models.

In [ ]:
class CustomHeatingModel(object):
    def __init__(self,heating_options):
        self.heating_options = heating_options
        self.logger = logging.getLogger(name=type(self).__name__)
    
    def calculate_event_properties(self,loop):
        self.number_events = self._calculate_number_events(loop)
        rates = self.constrain_distribution(loop)
        delays = (self.number_events*self.heating_options['frequency_parameter']
                  *self.cooling_time(loop)*rates/self.strand_energy(loop))
        running_total = 0.0
        rise_start = np.empty(rates.shape)
        for i in range(self.number_events):
            rise_start[i] = i*self.heating_options['duration'] + running_total
            running_total += delays[i]
        rise_end = rise_start + self.heating_options['duration_rise']
        decay_start = rise_end
        decay_end = rise_start + self.heating_options['duration']
        return {'magnitude':rates,
                'rise_start':rise_start,
                'rise_end':rise_end,
                'decay_start':decay_start,
                'decay_end':decay_end
               }
    
    def _calculate_number_events(self,loop):
        cooling_time = self.cooling_time(loop)
        return int(np.floor(self.base_config['total_time']
                            /(self.heating_options['duration']
                              + self.heating_options['frequency_parameter']*cooling_time)))
    
    def power_law(self,a0,a1,alpha,x):
        return ((a1**(alpha + 1.) - a0**(alpha + 1.))*x + a0**(alpha + 1.))**(1./(alpha + 1.))
    
    def strand_energy(self,loop):
        return ((self.heating_options['stress_level']*loop.field_strength.value.mean())**2)/8./np.pi
    
    def constrain_distribution(self,loop,max_tries=2000,tol=1e-3,**kwargs):
        # initial guess of bounds
        num_events = self._calculate_number_events(loop)
        strand_energy = self.strand_energy(loop)
        a0 = 0.5*strand_energy/num_events
        a1 = self.heating_options['delta_power_law_bounds']*a0
        # initialize parameters
        tries = 0
        err = 1.e+300
        best_err = err
        while tries < max_tries and err > tol:
            x = np.random.rand(num_events)
            h = self.power_law(a0, a1, self.heating_options['power_law_slope'], x)
            pl_sum = np.sum(h)
            chi = strand_energy/pl_sum
            a0 = chi*a0
            a1 = self.heating_options['delta_power_law_bounds']*a0
            err = np.fabs(1.-1./chi)
            if err < best_err:
                best = h
                best_err = err
            tries += 1

        self.logger.debug("chi = {}, # of tries = {}, error = {}".format(chi, tries, err))

        if tries >= max_tries:
            self.logger.warning("Power-law constrainer reached max # of tries, using best guess with error = {}".format(best_err))

        return np.array(random.sample(list(best), len(best)))
    
    def cooling_time(self,loop):
        half_length = loop.full_length.value/2.
        average_heating_rate_max = self.strand_energy(loop)/(self.heating_options['duration']/2.)#*u.erg/(u.cm**3)/u.s
        # set some constants
        alpha = -0.5
        chi = 6e-20#*(u.erg*(u.cm**3)/u.s*u.K**(0.5))
        kappa_0 = 1e-6#*(u.erg/u.cm/u.s*(u.K**(-7/2)))
        c1,c2,c3 = 2.0,0.9,0.6
        gamma = 5./3.
        # estimate max n0T0
        T0 = c2*(7.*half_length**2*average_heating_rate_max/2./kappa_0)**(2./7.)
        top_term = average_heating_rate_max - 2.*kappa_0*(T0**(3.5))/(7.*(c2**2.5)*c3*(half_length**2)*gamma)
        bottom_term = c1*chi*(T0**alpha)*(1. - c2/c3/gamma)
        n0 = np.sqrt(top_term/bottom_term)
        n0T0 = n0*T0
        # Cargill cooling expression
        term1 = (2. - alpha)/(1. - alpha)
        term2 = (kappa_0**(4. - 2.*alpha))*(chi**7)
        term3 = ((half_length)**(8. - 4.*alpha))/(n0T0**(3+2.*alpha))
        return term1*3.*const.k_B.cgs.value*(1/term2*term3)**(1/(11. - 2.*alpha))

In [ ]:
heating_options = {
    'duration':200.0,
    'delta_power_law_bounds':100.0,
    'duration_rise':100.0,
    'duration_decay':100.0,
    'stress_level':0.1,
    'frequency_parameter':0.1,
    'power_law_slope':-2.5
}